In [61]:
import numpy as np

# Colors
WHITE = 0
BLUE = 1
RED = 2
GREEN = 3  # Initial piece
COLORS = np.array([WHITE, BLUE, RED])

In [62]:
# Shapes
LONG4 = np.array((1, 1, 4))
RECTANGLE3 = np.array((2, 1, 3))
RECTANGLE2 = np.array((3, 1, 2))
SQUARE2 = np.array((2, 1, 2))
SHORT2 = np.array((1, 1, 2))
LONG1 = np.array((4, 1, 1))
RECTANGLE1 = np.array((3, 2, 1))
SQUARE1 = np.array((2, 2, 1))
SHORT1 = np.array((2, 1, 1))
SHAPES = np.array([LONG4, RECTANGLE3, RECTANGLE2, SQUARE2, SHORT2, LONG1,
                   RECTANGLE1, SQUARE1, SHORT1])

In [82]:
# Can we do that better ?
PIECES=np.column_stack((np.repeat(SHAPES, len(COLORS), axis=0), np.tile(COLORS, len(SHAPES))))
IDX, IDY, IDZ, IDCOL = 0, 1, 2, 3

In [84]:
ABC = (RECTANGLE1, False)
ACB = (RECTANGLE2, False)
BAC = (RECTANGLE1, True)
BCA = (RECTANGLE3, False)
CAB = (RECTANGLE2, True)
CBA = (RECTANGLE3, True)

In [89]:
CARD_WHITE = PIECES[PIECES[:,IDCOL] == WHITE]
CARD_BLUE = PIECES[PIECES[:,IDCOL] == BLUE]
CARD_RED = PIECES[PIECES[:,IDCOL] == RED]

CARD_LONG = PIECES[np.all(np.sort(PIECES[:,:3], axis=1)[:,:2] == np.array((1, 1)), axis=1)]
CARD_SQUARE = PIECES[np.all(np.sort(PIECES[:,:3], axis=1)[:,1:] == np.array((2, 2)), axis=1)]
CARD_RECTANGLE = PIECES[np.all(np.sort(PIECES[:,:3], axis=1) == np.array((1, 2, 3)), axis=1)]

#CARD_RECTANGLE = frozenset(p for p in PIECES if sorted(p.shape.value) == [1, 2, 3])

In [ ]:
INITIAL_DECK = np.array([CARD_SQUARE, CARD_WHITE, CARD_LONG, CARD_RECTANGLE, CARD_BLUE,
                         CARD_RED, CARD_WHITE, CARD_RECTANGLE, CARD_WHITE, CARD_BLUE,
                         CARD_RED, CARD_SQUARE, CARD_LONG, CARD_RECTANGLE, CARD_BLUE,
                         CARD_LONG, CARD_LONG, CARD_BLUE, CARD_SQUARE, CARD_RED, CARD_LONG,
                         CARD_BLUE, CARD_RED, CARD_LONG, CARD_RECTANGLE, CARD_SQUARE,
                         CARD_WHITE, CARD_BLUE, CARD_WHITE, CARD_WHITE, CARD_RED, CARD_RED,
                         CARD_RECTANGLE, CARD_RECTANGLE, CARD_SQUARE, CARD_SQUARE])

CARDS_NUM = len(INITIAL_DECK)

In [100]:
BOARD = np.array((6, 4))

In [188]:
def rotate(shape):
    return shape[[1, 0, 2]]
    

In [184]:
import matplotlib.pyplot as plt
import numpy as np

# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    
    
def draw(pieces):
    # shape for pieces is (n, 7), 7 being (l, w, h, c, x, y, z)
    idl, idw, idh, idc, idx, idy, idz = range(7)
    max_height = np.max(a[:,idh] + a[:,idz]) + 1
    
    # prepare some coordinates
    shape = np.concatenate((BOARD, np.array((max_height,))))
    x, y, z = np.indices(shape)
    colors = np.full(shape, "")
    color_names = ["white", "blue", "red", "green"]
    
    base = (z == 0)
    colors[base] = 'grey'
    pieces[:,idz] += 1
    
    for piece in pieces:
        voxels = ((piece[idx] <= x) & (x < piece[idx] + piece[idl]) &
                  (piece[idy] <= y) & (y < piece[idy] + piece[idw]) &
                  (piece[idz] <= z) & (z < piece[idz] + piece[idh]))

        colors[voxels] = color_names[piece[idc]]

    voxels = colors != ""

    # and plot everything
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.set_aspect('equal')
    ax.voxels(voxels, facecolors=colors, edgecolor='#00000020')
    set_axes_equal(ax)
    plt.axis('off')
    plt.show()



In [276]:
from typing import Sequence
import numpy as np

class Game:
    def __init__(
        self,
        turn: int,
        deck,
        used,
        board_cubes,
        board_pieces,
        first,
    ):
        assert turn < CARDS_NUM
        self.turn = turn
        self.deck = deck
        self.used = used
        self.board_cubes = board_cubes
        self.board_pieces = board_pieces
        self.first = first

    @classmethod
    def new(cls, first, cards_order=None):
        first_shape, rotated = first
        if rotated:
            first_shape = first_shape[[1, 0, 2]]
        
        sorted_cards = np.arange(CARDS_NUM)
        if cards_order:
            assert np.sort(cards_order) == sorted_cards
        else:
            cards_order = sorted_cards    
        deck = INITIAL_DECK[cards_order]
        
        board_cubes = np.zeros(BOARD, dtype=int)
        board_cubes[:first_shape[IDX],:first_shape[IDY]] = first_shape[IDZ]
        board_pieces = np.concatenate([first_shape, (GREEN, 0, 0, 0)])
        return cls(turn=0, deck=deck, used=np.empty((0, 1)), board_cubes=board_cubes, board_pieces=board_pieces, first=first)
    
    def draw(self):
        draw(pieces=self.board_pieces)
    
    def possible_moves(self):
        authorized = self.deck[self.turn] 
        available = authorized[np.all(np.any(authorized != used[:, None, :], axis=2), axis=0)]
        
        avail_and_rotated = np.unique(np.concatenate((available, available[:,(1, 0, 2, 3)])), axis=0)
        
        all_moves = combine(avail_and_rotated, np.arange(BOARD[0]), np.range(BOARD[1]))
        
        return all_moves
        

In [ ]:

    def check(self):
        piece = self.piece
        pos = self.pos
        piece_cubes = self.piece_cubes
        piece_shape = self.piece_shape
            
        # Is piece out of bounds:
        if any(p < 0 for p in pos):
            raise OutOfBounds("Invalid position (<0)")
        if any(p + s > b for p, s, b in zip(pos, piece_shape, BOARD)):
            raise OutOfBounds("Invalid position (>border)")
        
        # Check all heights under piece: should be all equal to z coord -1
        z = self.pos3.z
        
        if z == 0:
            to_check = [C2(pos.x - 1, pos.y),
                        C2(pos.x, pos.y - 1),
                        C2(pos.x + piece_shape.x, pos.y),
                        C2(pos.x, pos.y + piece_shape.y)]
            for c in to_check:
                try:
                    if self.game.board_cubes[c]:
                        break
                except IndexError:
                    continue
            else:
                raise PieceNotNextToAnotherPiece()
        
        all_z = self.game.board_cubes[piece_cubes]
        if not np.all(all_z == (z)):
            raise PieceNotOnOtherPieces(f"Wrong height (should be all {z}):\n{all_z}")
        
        # Is piece next to similar piece
        # Check 5 directions with board_pieces
        forbidden = {C3(pos.x, pos.y, z - piece_shape.z),
                     C3(pos.x - piece_shape.x, pos.y, z),
                     C3(pos.x + piece_shape.x, pos.y, z),
                     C3(pos.x, pos.y - piece_shape.y, z),
                     C3(pos.x, pos.y + piece_shape.y, z)}

        if any((f, piece_shape) in self.game.board_pieces for f in forbidden):
            raise PieceNextToSimilarPiece()

In [339]:
import functools

def concatenate_two(a, b):
    return np.hstack((np.repeat(a, len(b), axis=0), np.tile(b, (len(a), 1))))

def combine(*arrays):
    return functools.reduce(concatenate_two, arrays)

In [186]:
class Move:
    def __init__(self, game:Game, piece: Piece, rotated: bool, pos:C2):
        self.game = game
        self.piece = piece
        self.rotated = rotated
        self.pos = pos
        self.pos3 = C3(*pos, self.game.board_cubes[pos])
        
        self.piece_shape = piece.shape.value
        if rotated:
            self.piece_shape = rotate(self.piece_shape)
        
        x, y = np.indices(BOARD)
        self.piece_cubes = ((pos.x <= x) & (x < pos.x + self.piece_shape.x) &
                            (pos.y <= y) & (y < pos.y + self.piece_shape.y))
    
    def __repr__(self):
        return f"{self.piece} {self.pos} {'r' if self.rotated else ''}"
    
    def check(self):
        piece = self.piece
        pos = self.pos
        piece_cubes = self.piece_cubes
        piece_shape = self.piece_shape
        # Is piece available
        if piece not in PIECES - frozenset(self.game.used):
            raise PieceNotAvaliable("Piece not available")
        # Is piece valid given card
        if piece not in self.game.card.value:
            raise PieceNotIncludedInCare("Forbidden piece given current card")
            
        # Is piece out of bounds:
        if any(p < 0 for p in pos):
            raise OutOfBounds("Invalid position (<0)")
        if any(p + s > b for p, s, b in zip(pos, piece_shape, BOARD)):
            raise OutOfBounds("Invalid position (>border)")
        
        # Check all heights under piece: should be all equal to z coord -1
        z = self.pos3.z
        
        if z == 0:
            to_check = [C2(pos.x - 1, pos.y),
                        C2(pos.x, pos.y - 1),
                        C2(pos.x + piece_shape.x, pos.y),
                        C2(pos.x, pos.y + piece_shape.y)]
            for c in to_check:
                try:
                    if self.game.board_cubes[c]:
                        break
                except IndexError:
                    continue
            else:
                raise PieceNotNextToAnotherPiece()
        
        all_z = self.game.board_cubes[piece_cubes]
        if not np.all(all_z == (z)):
            raise PieceNotOnOtherPieces(f"Wrong height (should be all {z}):\n{all_z}")
        
        # Is piece next to similar piece
        # Check 5 directions with board_pieces
        forbidden = {C3(pos.x, pos.y, z - piece_shape.z),
                     C3(pos.x - piece_shape.x, pos.y, z),
                     C3(pos.x + piece_shape.x, pos.y, z),
                     C3(pos.x, pos.y - piece_shape.y, z),
                     C3(pos.x, pos.y + piece_shape.y, z)}

        if any((f, piece_shape) in self.game.board_pieces for f in forbidden):
            raise PieceNextToSimilarPiece()

    def play(self):
        # Add piece_cubes to board_cubes
        self.game.board_cubes[self.piece_cubes] += self.piece_shape.z
        # Add pos & piece to board_pieces 
        self.game.board_pieces[(self.pos3, self.piece_shape)] = self.piece.color
        
        self.game.used.append(self.piece)
        self.game.turn += 1

IndentationError: expected an indented block (<ipython-input-186-b8a99bb1c34d>, line 16)

array([[1, 1, 4, 1],
       [1, 1, 4, 2],
       [2, 1, 3, 0],
       [2, 1, 3, 1],
       [2, 1, 3, 2],
       [3, 1, 2, 0],
       [3, 1, 2, 1],
       [3, 1, 2, 2],
       [2, 1, 2, 0]])

In [192]:
len(np.array([[1, 2], [3, 4]]))

2

In [6]:
import random
def random_play():
    g = Game.new(FirstPos.ABC)
    while True:
        try:
            next(g.possible_moves()).play()
        except StopIteration:
            break


In [32]:
%timeit tuple(random_play())

613 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%prun random_play()